In [ ]:
from haystack.utils import  convert_files_to_dicts, print_answers
from haystack.nodes import FARMReader
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.pipelines import ExtractiveQAPipeline

In [ ]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

In [ ]:
# doc_dir = "Data/article_txt_got"
# s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
# fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

In [ ]:
doc_dir = "../../Site Data/Data"
dicts = convert_files_to_dicts(dir_path=doc_dir,split_paragraphs=True)
document_store.write_documents(dicts)

In [ ]:
retriever = DensePassageRetriever(document_store=document_store,
                                 query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
                                 passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
                                 max_seq_len_query=64,
                                 max_seq_len_passage=256,
                                 batch_size=16,
                                 use_gpu=True,
                                 embed_title=True,
                                 use_fast_tokenizers=True)
document_store.update_embeddings(retriever)

In [ ]:
reader = FARMReader(model_name_or_path="deepset/roberta-large-squad2", use_gpu=True)

In [ ]:
pipeline = ExtractiveQAPipeline(reader, retriever)

In [22]:
prediction = pipeline.run(query="Where is mits located?",
                         params = {"Retriever":{"top_k":10}, 
                                  "Reader":{"top_k":10}})

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.40 Batches/s]


In [23]:
print_answers(prediction,details="minimum")


Query: Where is mits located?
Answers:
[   {   'answer': 'Ernakulam',
        'context': 'ering (CSE) at the Muthoot Institute of Technology and '
                   'Science (MITS), Ernakulam! As you glance through the '
                   'faculty profile, I am sure that you will a'},
    {   'answer': 'Puthencruz',
        'context': 'Department profile Muthoot Institute of Technology and '
                   'Science (MITS) Puthencruz was established in May 2013 and '
                   'started its academic program in Augus'},
    {   'answer': 'Kochi',
        'context': 'pal Muthoot Institute of Technology & Science Varikoli '
                   'P.O., Puthencruz, Kochi – 682308, Ernakulam District. Ph. '
                   '0484 – 2732100, 2732111, 2733011 Fax:'},
    {   'answer': 'Varikoli overlooking the Kochi-Madurai National Highway',
        'context': '7 kms from the MITS Campus. MITS is situated at Varikoli '
                   'overlooking the Kochi-Madurai National H